# 01-01: Data Preparation

The basic idea is to create a sequential recommendation system in Keras by combining Convolutional Neural Networks (CNNs) with Recurrent Neural Networks (RNNs) for sequence prediction.

Before applying deep learning models, preprocessing the sequence data is necessary. This involves tokenizing the sequences and possibly using techniques like padding to ensure that all input sequences have the same length. For your dataset, you might consider each session as a sequence and each item interaction within a session as a token within that sequence.

In [1]:
import os
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/usr/local/cuda-12.2"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import pandas as pd
from pprint import pprint
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Dense
from keras.optimizers import Adam

import tensorflow as tf
import tensorflow_recommenders as tfrs

2024-03-01 12:32:18.243342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 12:32:18.243368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 12:32:18.244237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 12:32:18.248369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 12:32:18.694013: W tensorflow/compiler/tf2

## 1. Dataset Description

- Describe the dataset in detail: sources, features, and statistics.
- Discuss any limitations or biases present in the dataset.

In [3]:
data_path = '../../data/coveo'
model_path = '../../models/coveo'

# file paths for the data files
train_path = f'{data_path}/coveo_processed_view_train_tr.tsv'
validation_path = f'{data_path}/coveo_processed_view_train_valid.tsv'
test_path = f'{data_path}/coveo_processed_view_test.tsv'

In [4]:
# load the datasets
df_train = pd.read_csv(train_path, sep='\t')
df_validation = pd.read_csv(validation_path, sep='\t')
df_test = pd.read_csv(test_path, sep='\t')

In [5]:
# show the shape of the datasets
print('Train      :', df_train.shape)
print('Validation :', df_validation.shape)
print('Test       :', df_test.shape)

Train      : (1368003, 3)
Validation : (43032, 3)
Test       : (52501, 3)


In [6]:
# head of the training set
display(df_train.head())

,SessionId,ItemId,Time
0,2,1,1544909035879
1,2,2,1544909231588
2,2,3,1544909334734
3,5,4,1545550811488
4,5,5,1545550910607


In [7]:
# unique sessions in each dataset
print('Train      :', df_train.SessionId.nunique())
print('Validation :', df_validation.SessionId.nunique())
print('Test       :', df_test.SessionId.nunique())

Train      : 159766
Validation : 5905
Test       : 7748


In [8]:
# average session length in each dataset
print('--- Train ---')
print(df_train.groupby('SessionId').size().describe())
print('--- Validation ---')
print(df_validation.groupby('SessionId').size().describe())
print('--- Test ---')
print(df_test.groupby('SessionId').size().describe())

--- Train ---
count    159766.000000
mean          8.562541
std          10.909689
min           2.000000
25%           3.000000
50%           5.000000
75%          10.000000
max         454.000000
dtype: float64
--- Validation ---
count    5905.000000
mean        7.287384
std         8.974551
min         2.000000
25%         3.000000
50%         4.000000
75%         8.000000
max       134.000000
dtype: float64
--- Test ---
count    7748.000000
mean        6.776071
std         8.341588
min         2.000000
25%         2.000000
50%         4.000000
75%         8.000000
max       146.000000
dtype: float64


## 2. Data Preprocessing

- Detail the preprocessing steps with explanations for each decision.
- Show visualizations of data distribution, session lengths, and item frequencies.

In [9]:
# convert the items ids to strings
df_train['ItemId'] = df_train['ItemId'].astype(str)
df_validation['ItemId'] = df_validation['ItemId'].astype(str)
df_test['ItemId'] = df_test['ItemId'].astype(str)

### 2.1 Sequence Creation

The first step involves creating sequences of item interactions for each session. This requires grouping the data by SessionId and ordering it within each group based on the Time column. Each sequence represents a series of item interactions within a session.

In [10]:
# Sort by SessionId and Time to ensure the order is correct
df_train_sorted = df_train.sort_values(by=['SessionId', 'Time'])
df_validation_sorted = df_validation.sort_values(by=['SessionId', 'Time'])
df_test_sorted = df_test.sort_values(by=['SessionId', 'Time'])

# Create sequences of ItemIds grouped by SessionId
train_sequences = df_train_sorted.groupby('SessionId')['ItemId'].apply(list)
validation_sequences = df_validation_sorted.groupby('SessionId')['ItemId'].apply(list)
test_sequences = df_test_sorted.groupby('SessionId')['ItemId'].apply(list)

In [11]:
train_sequences.head(5)

SessionId
2                                             [1, 2, 3]
5                                                [4, 5]
13                                             [10, 11]
14                                 [12, 13, 14, 15, 16]
18    [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 3...
Name: ItemId, dtype: object

### 2.2 Tokenization (Categorical Features Encoding)

We need to ensure that ItemIds are treated as categorical inputs.

Create a tokenizer to encode ItemIds as integers, 0 and 1 are special values, where 0 should be for padding and 1 for out of vocabulary items.

In [12]:
# get a list of the unique item ids across all datasets
unique_items = pd.concat([df_train, df_validation, df_test]).ItemId.unique()

# use keras to map the item ids to a sequential list of integer values,
# 0 should be for padding and 1 for out of vocabulary items
tokenizer = Tokenizer(num_words=len(unique_items) + 2, oov_token=1)
tokenizer.fit_on_texts(unique_items)

# save the tokenizer
tokenizer_path = f'{model_path}/item_id_tokenizer.json'
with open(tokenizer_path, 'w') as file:
    file.write(tokenizer.to_json())

In [13]:
# tokenize the sequences
train_sequences_tokenized = tokenizer.texts_to_sequences(train_sequences)
validation_sequences_tokenized = tokenizer.texts_to_sequences(validation_sequences)
test_sequences_tokenized = tokenizer.texts_to_sequences(test_sequences)

In [14]:
train_sequences_tokenized[:5]

[[2, 3, 4],
 [5, 6],
 [7, 8],
 [9, 10, 11, 12, 13],
 [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 24, 27, 28]]

### 2.3 Padding

To handle sessions of varying lengths, we'll need to pad the sequences so that they all have the same length, making them suitable for batch processing.

In [15]:
# use the last item as the target and the rest as the input
def split_input_target(sequence):
    return sequence[:-1], sequence[-1]

train_sequences_input = list(map(split_input_target, train_sequences_tokenized))
validation_sequences_input = list(map(split_input_target, validation_sequences_tokenized))
test_sequences_input = list(map(split_input_target, test_sequences_tokenized))

In [16]:
train_sequences_input[:5]

[([2, 3], 4),
 ([5], 6),
 ([7], 8),
 ([9, 10, 11, 12], 13),
 ([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 24, 27], 28)]

In [17]:
# separate into input and target arrays
train_input, y_train = map(list, zip(*train_sequences_input))
validation_input, y_validation = map(list, zip(*validation_sequences_input))
test_input, y_test = map(list, zip(*test_sequences_input))

In [18]:
pprint(train_input[:5])
print('-'*10)
pprint(y_train[:5])

[[2, 3],
 [5],
 [7],
 [9, 10, 11, 12],
 [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 24, 27]]
----------
[4, 6, 8, 13, 28]


In [19]:
# Determine the maximum sequence length for padding
max_sequence_length = max(map(len, train_input))

# pad the sequences
X_train = pad_sequences(train_input, maxlen=max_sequence_length, padding='post')
X_validation = pad_sequences(validation_input, maxlen=max_sequence_length, padding='post')
X_test = pad_sequences(test_input, maxlen=max_sequence_length, padding='post')

In [20]:
X_train[:5]

array([[ 2,  3,  0, ...,  0,  0,  0],
       [ 5,  0,  0, ...,  0,  0,  0],
       [ 7,  0,  0, ...,  0,  0,  0],
       [ 9, 10, 11, ...,  0,  0,  0],
       [14, 15, 16, ...,  0,  0,  0]], dtype=int32)

## 3. Model Architecture

In [21]:
model = Sequential()

2024-03-01 12:32:23.043517: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-01 12:32:23.070367: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### 3.1 Embedding Layers

Embedding layers transform sparse, integer-encoded data into dense vectors of fixed size. This is particularly useful for categorical data with many categories, where one-hot encoding would result in vectors that are mostly zeros and inefficient both in terms of memory and computation. Embeddings capture more information per byte and allow the model to learn relationships between categories (in this case, ItemIds).

**Why It Is Done:**

Dimensionality Reduction: Embeddings reduce the dimensionality of the categorical data, representing items in a more compact and dense vector space.
Semantic Representation: The dense vectors obtained from embeddings can capture semantic relationships between items, such as similarity in user interaction patterns, which is not possible with one-hot encoding or similar techniques.
Improved Learning: By learning an optimal representation of items in a lower-dimensional space, embeddings can improve the model's performance on the recommendation task.
Implementation

In [22]:
max_item_id = max(tokenizer.index_word.keys())  # get the maximum item id from the tokenizer
embedding_dim = 64                              # Size of the embedding vectors

# Embedding layer for vectorizing ItemIds into dense vectors
model.add(Embedding(input_dim=max_item_id + 1, output_dim=embedding_dim, input_length=max_sequence_length))

## 3.2 CNN Layers

CNNs, particularly 1D CNNs (Conv1D), are adept at recognizing local patterns within sequences. The key advantage of using CNNs in sequence data processing is their ability to apply the same transformation across different parts of a sequence, making them translation invariant. This means that if a CNN learns a particular pattern (e.g., a specific sequence of item interactions) at one point in a session, it can recognize the same pattern at a different point, enhancing the model's ability to understand user behavior across sessions.

**Why It Is Done:**

- **Local Pattern Recognition**: CNNs excel at identifying patterns within localized regions of the input data. For sequence data, this means recognizing specific sequences of item interactions that are indicative of broader user preferences.
Translation Invariance: The ability of CNNs to recognize patterns regardless of their position in the sequence is particularly valuable for session-based recommendation systems, where the order of interactions can vary significantly across sessions.

- **Efficiency and Scalability**: CNNs can process data more efficiently than fully connected networks, especially for large datasets. This efficiency makes them suitable for handling the vast amounts of data typically involved in recommendation systems.

In [23]:
# Conv1D + MaxPooling1D for local pattern recognition and dimensionality reduction
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(5))

# Additional Conv1D + GlobalMaxPooling1D for deeper pattern recognition
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
#model.add(GlobalMaxPooling1D())

## 3.3  Recurrent Neural Network Layers

Incorporating Recurrent Neural Networks (RNNs) for sequence modeling into a deep learning architecture that already includes Convolutional Neural Networks (CNNs) layers is a strategic approach to handle sequence data effectively, especially for tasks like sequential recommendation systems. This combination leverages the strengths of both CNNs and RNNs, where CNNs excel in extracting local and position-invariant features, and RNNs are adept at capturing temporal or sequential dependencies.

The primary purpose of incorporating RNN layers after CNN layers is to model the temporal or sequential dependencies in the data. While CNNs are excellent for reducing a long sequence into a shorter one with extracted features (as mentioned in the document context about processing long sequences with CNNs before RNNs), they do not inherently understand the order or sequence of these features. RNNs, with their internal state and loop mechanism, can take the sequence of extracted features and learn patterns over time, making them ideal for tasks where the sequence order matters, such as predicting the next item in a session.

The CNN layers act as a preprocessing step that transforms the raw sequence data into a higher-level, downsampled representation. This transformation is crucial because it allows the RNN layers to process the data more efficiently by focusing on the most relevant features extracted by the CNNs. The RNN can then model the temporal dynamics of these features across the sequence. This setup is particularly beneficial for handling sequences that are too long to be processed directly by RNNs, as it reduces the sequence length and complexity before reaching the RNN layers.

RNNs are specifically designed to handle sequence data, making them a natural choice for sequence modeling tasks. They can maintain information about previous states or inputs through their hidden states, allowing them to make predictions based on the entire history of a sequence. This capability is crucial for understanding the context in sequential recommendation systems, where the relevance of an item can depend on the items that were interacted with previously.

In [24]:
model.add(LSTM(32))  # LSTM layer with 32 units. Adjust the number of units based on your dataset and complexity.

## 3.4 Output Layer

The output layer of a sequential recommendation system is crucial as it directly influences the system's ability to make accurate and relevant recommendations. The design of this layer depends on the specific task at hand – in the case of a sequential recommendation system, the task is typically to predict the next item(s) a user is likely to interact with, based on their previous interactions within a session.

Given the sequential nature of the data, where the order of interactions matters significantly, the output layer must capture and utilize the rich representations learned by the preceding layers (embedding, CNNs for local pattern recognition, and RNNs for capturing temporal dynamics) to make these predictions.

### How It Relates to Preceding Layers:
The output layer receives processed and transformed data from the previous layers, which have worked in tandem to encode the input sequences into a meaningful representation that captures both the content and context of user interactions. The embedding layer first converts item IDs into dense vectors, capturing similarity and relationships between items. The CNN layers then extract local patterns within these embeddings, and the RNN layers model the sequential dependencies. The output layer must leverage this rich representation to make accurate predictions.

### Output Layer Design for Recommendation System:
For a sequential recommendation system, the output layer often uses a softmax activation function. This function is suitable because it can output a probability distribution over the set of possible items, indicating the likelihood of each item being the next interaction. This approach requires the output layer to have as many units as there are unique items in the dataset, with each unit corresponding to a specific item.

### Practical Use in Recommendation Systems:
In practice, the output of the model can be used to rank items based on their predicted likelihood of being the next user interaction. The system can then recommend the top-N items with the highest probabilities. This method allows for personalized recommendations that adapt based on the user's current session activities.

In [25]:
# Output layer for predicting the next item in the sequence
model.add(Dense(max_item_id+1, activation='softmax'))

## 3.5 Evaluation Metrics

It is important to establish a robust evaluation protocol and optimizing model performance, we'll focus on implementing and optimizing for the metrics Recall@K and MRR@K (Mean Reciprocal Rank at K). These metrics are particularly relevant for recommendation systems and ranking problems, where the goal is to accurately predict and rank a set of items in order of relevance.

### Evaluation Metrics: Recall@K and MRR@K

**Recall@K** measures the fraction of relevant items that are retrieved among the top-K recommendations. It's crucial for assessing the model's ability to capture the most relevant items within the first K suggestions.

**MRR@K** (Mean Reciprocal Rank at K) focuses on the rank of the first relevant item, averaging this metric over multiple queries. It's valuable for understanding how well the model ranks the first relevant item, emphasizing the importance of items ranked closer to the top.

Rational for Choosing Recall@K and MRR@K

- Recall@K is essential for ensuring that the top-K recommendations contain items truly relevant to the user, enhancing user satisfaction and engagement.
- MRR@K complements Recall@K by ensuring that not only are relevant items included in the top-K recommendations, but they are also ranked as high as possible.
Implementation

Given the emphasis on hyperparameter optimization and model evaluation, we'll implement custom functions for Recall@K and MRR@K and integrate them into the model training and evaluation process. This approach allows us to directly optimize and evaluate the model based on metrics that align closely with our objectives.

In [26]:
# def recall_at_k(y_true, y_pred, k=10):
#     # Assuming y_true and y_pred are batch-wise lists of true labels and predicted labels, respectively
#     top_k_preds = K.np.argsort(y_pred)[:, -k:]  # Get indices of top-k predictions
#     relevant_items = K.sum(K.any(K.in1d(top_k_preds, y_true), axis=1))  # Count relevant items in top-k predictions
#     return relevant_items / K.cast(K.shape(y_true)[0], K.floatx())  # Compute recall

# def mrr_at_k(y_true, y_pred, k=10):
#     top_k_preds = K.np.argsort(y_pred)[:, -k:]
#     ranks = K.np.argmax(K.in1d(top_k_preds, y_true), axis=1) + 1  # Find ranks of true items
#     ranks[ranks > k] = 0  # Set ranks > k to 0 as they are not found within top-k
#     return K.mean(1.0 / ranks[ranks > 0])  # Compute mean reciprocal rank

In [27]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy')
              #metrics=[tf.metrics.recall_at_k])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 453, 64)           695680    
                                                                 
 conv1d (Conv1D)             (None, 449, 128)          41088     
                                                                 
 max_pooling1d (MaxPooling1  (None, 89, 128)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 85, 128)           82048     
                                                                 
 lstm (LSTM)                 (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 10870)             358710    
                                                        

## 4. Model Training

In [28]:
# Train the model
history = model.fit(X_train, to_categorical(y_train, max_item_id+1),
                    epochs=10,  # Number of epochs to train for
                    batch_size=128,  # Batch size
                    validation_data=(X_validation, to_categorical(y_validation, max_item_id+1)))

Epoch 1/10
1064/1249 [========================>.....] - ETA: 10s - loss: 8.8149